In [1]:
import polars as pl
import time
from connDb import path_db

In [2]:
def getRedeSocialStg() -> pl.LazyFrame:
    """ Lendo os dados da tabela  stg_deputados e  filtrando as redes  sociais  
        nulas (5 é  um valor arbitrário, com 2 já é possível filtrar os nulos)
    """
    df = (
            pl.read_database_uri('SELECT "id", "redeSocial" from stg_deputados', path_db)
            .filter(pl.col("redeSocial")
            .str.len_bytes() > 5)
            .lazy()
    )

    return df

In [3]:

def transformRedeSocial(df: pl.LazyFrame) -> pl.DataFrame:
    """ Normalizando a coluna de Redes Sociais removendo caracteres especiais e divid
        -indo as urls por ','. Depois adicionando um contador que é utilizado em   se
        -guida para repetir o Id em uma lista correspondente às urls. Depois  explodi
        -mos as duas listas, resultando em uma tabela de mapeamento, onde cada url de
        rede social corresponde a um Id de deputado.
    """
    df = (
            df.with_columns(
                pl.col("redeSocial")
                .str.replace_all(r'[\[\]"]', '')
                .str.split(", ")
            )
            .with_columns(
                count=pl.col("redeSocial")
                .list.len()
            )
            .select(
                pl.col("id").repeat_by(pl.col("count")).alias("id_deputado"),
                pl.col("redeSocial")
            )
            .explode("id_deputado", "redeSocial")
            .with_columns(
                pl.col("id_deputado").cast(pl.Int64)
            )
            .collect()
    )
    
    return df


In [4]:
def loadRedeSocial(df:pl.DataFrame):
    """ Carregando os dados tratados na tabela final 'rede_social', apta para ser
        utilizada em análises futuras. 
    """
    try:
        df.write_database("rede_social", path_db, if_exists="replace", engine="sqlalchemy")
    
    except:
        print("Erro ao carregar dados na tabela 'rede_social'\n")
        time.sleep(1)
    else:
        print("Sucesso ao carregar dados na tabela 'rede_social'\n")

In [5]:
df = getRedeSocialStg()

new_df = transformRedeSocial(df)

loadRedeSocial(new_df)



Sucesso ao carregar dados na tabela 'rede_social'

